In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.io import loadmat, savemat
import h5py
from spectral import *
from functions import *
import warnings
import time
import cv2

/usr/local/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
file_names = ['p011_neutral_right','p011_smile_right','p015_neutral_right']
VIS_dict = {}
NIR_dict = {}
RGB_dict = {}
for file_name in file_names:
    file = h5py.File("./dataset/Hyper-skin/NIR/{}.mat".format(file_name), 'r')
    NIR_dict[file_name] = np.transpose(np.array(file['cube']), [2, 1, 0])
    file.close()
    RGB_dict[file_name] = plt.imread("./dataset/Hyper-skin/RGB_CIE/{}.jpg".format(file_name))

/tmp/ipykernel_936108/329583714.py:7: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  NIR_dict[file_name] = np.transpose(np.array(file['cube']), [2, 1, 0])


In [3]:
file_names = ['p011_neutral_right','p011_smile_right','p015_neutral_right']
data_dict = {}
new_RGB_dict = {}
data_dict['p011_neutral_right'] = NIR_dict['p011_neutral_right'][600:800,200:400,-13:-5]
data_dict['p011_smile_right'] = NIR_dict['p011_smile_right'][190:375,400:600,-13:-5]
data_dict['p015_neutral_right'] = NIR_dict['p015_neutral_right'][600:800,250:450,15:25]
new_RGB_dict['p011_neutral_right'] = RGB_dict['p011_neutral_right'][600:800,200:400,:]
new_RGB_dict['p011_smile_right'] = RGB_dict['p011_smile_right'][190:375,400:600,:]
new_RGB_dict['p015_neutral_right'] = RGB_dict['p015_neutral_right'][600:800,250:450,:]

In [4]:
savemat('./LARTVAD/p011_neutral.mat', {'data': data_dict['p011_neutral_right']})
savemat('./LARTVAD/p011_smile.mat', {'data': data_dict['p011_smile_right']})
savemat('./LARTVAD/p015_neutral.mat', {'data': data_dict['p015_neutral_right']})

In [5]:
approach_list = ['LARTVAD']
for approach in approach_list:
    i = 0
    exec(approach + '_dict={}')
    exec(approach + '_time_dict={}')
    exec(approach + '_AS=loadmat("./LARTVAD/'+ approach +'__hyper_skin.mat")["'+ approach +'_AS"]["as"][0]')
    exec(approach + '_time=loadmat("./LARTVAD/'+ approach +'__hyper_skin.mat")["'+ approach +'_AS"]["time"][0]')
    for file_name in file_names:
        exec(approach + '_dict["' + file_name + '"] = '+ approach + '_AS[{}]'.format(i))
        exec(approach + '_time_dict["' + file_name + '"] = float('+ approach + '_time[{}])'.format(i))
        i = i + 1

<string>:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)


In [12]:
rxval_dict = {}
lev_dict = {}
lev_fast_dict = {}
lev_min_dict = {}
lev_fast_min_dict = {}

lev_time_dict = {}
lev_fast_time_dict = {}
order = [1,1]
for file_name in file_names:
    HSI = data_dict[file_name]
    img = np.mean(HSI,axis=2)
    rxval = rx(HSI)

    start = time.time()
    lev = two_dim_AR_lev(img, order)
    #lev = two_dim_AR_lev_iteration(img, order)
    end = time.time()
    lev_time_dict[file_name] = end - start

    start_fast = time.time()
    lev_fast = fast_2d_AR_leverage_score(img, order, subsample_size = 100)
    end_fast = time.time()
    lev_fast_time_dict[file_name] = end_fast - start_fast
    
    lev_min = lev_adjust(lev, order, method = 'min')
    lev_fast_min = lev_adjust(lev_fast, order, method = 'min')


    rxval_dict[file_name] = rxval / np.max(rxval)
    lev_dict[file_name] = lev
    lev_min_dict[file_name] = lev_min
    lev_fast_dict[file_name] = lev_fast
    lev_fast_min_dict[file_name] = lev_fast_min

In [7]:
lev_dict = lev_min_dict
lev_fast_dict = lev_fast_min_dict

In [14]:
lev_dict['p011_smile_right'] = cv2.fastNlMeansDenoising((lev_dict['p011_smile_right']*255 / np.max(lev_dict['p011_smile_right'])).astype('uint8'), None, 10, 7, 21)
lev_dict['p011_neutral_right'] = cv2.fastNlMeansDenoising((lev_dict['p011_neutral_right']*255 / np.max(lev_dict['p011_neutral_right'])).astype('uint8'), None, 5, 7, 21)
lev_dict['p015_neutral_right'] = cv2.fastNlMeansDenoising((lev_dict['p015_neutral_right']*255 / np.max(lev_dict['p015_neutral_right'])).astype('uint8'), None, 20, 7, 21)

lev_fast_dict['p011_smile_right'] = cv2.fastNlMeansDenoising((lev_fast_dict['p011_smile_right']*255 / np.max(lev_fast_dict['p011_smile_right'])).astype('uint8'), None, 10, 7, 21)
lev_fast_dict['p011_neutral_right'] = cv2.fastNlMeansDenoising((lev_fast_dict['p011_neutral_right']*255 / np.max(lev_fast_dict['p011_neutral_right'])).astype('uint8'), None, 5, 7, 21)
lev_fast_dict['p015_neutral_right'] = cv2.fastNlMeansDenoising((lev_fast_dict['p015_neutral_right']*255 / np.max(lev_fast_dict['p015_neutral_right'])).astype('uint8'), None, 20, 7, 21)

In [15]:
L = len(file_names)
approach_list = ['rxval', 'LARTVAD', 'lev', 'lev_fast']
app_num = len(approach_list) + 1
fig, ax = plt.subplots(nrows=L, ncols=app_num, figsize=(20, 12), dpi=800, sharex=True, sharey=True)
i = 0
for file_name in file_names:
    j = 1
    plt.subplot(L,app_num,i*app_num + j)
    if(i==0):
        plt.title('Pseudo Color')
    plt.imshow(new_RGB_dict[file_name])
    plt.axis('off')
    plt.xticks([])
    plt.yticks([])
    j = j + 1

    for approach in approach_list:
        plt.subplot(L,app_num,i*app_num + j)
        if(i==0):
            if(approach == 'rxval'):
                plt.title('RX')
            if(approach == 'lev'):
                plt.title('LEV-exact')
            if(approach == 'lev_fast'):
                plt.title('LEV-appr')
            if(approach == 'LARTVAD'):
                plt.title('LARTVAD')
        exec('plt.imshow(' + approach + '_dict["' + file_name + '"])')
        plt.axis('off')
        plt.xticks([])
        plt.yticks([])
        j = j + 1
    i = i + 1
plt.subplots_adjust(wspace=-0.1,hspace=0.05)
plt.show()